<a href="https://colab.research.google.com/github/ZaraGiraffe/ZaraNLP/blob/main/text_generation_by_characters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
from torch import nn

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.data import Dataset

import numpy as np

import string
from time import time

# Get the data file

In [12]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [13]:
all_text = open(path_to_file, "r").read()
len(all_text)

1115394

In [82]:
vocab_length = len(set(all_text))
vocab_length

65

# My first Try to predict next character in tensorflow

Creater helper transforms

In [89]:
ids_from_chars = layers.StringLookup(vocabulary=list(set(all_text)))

In [90]:
exm_ids = ids_from_chars(tf.strings.unicode_split(["avc", "vvc"], input_encoding="UTF-8"))
exm_ids

<tf.RaggedTensor [[17, 19, 12],
 [19, 19, 12]]>

In [91]:
chars_from_ids = layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [92]:
exm_chars = chars_from_ids(exm_ids)
exm_chars

<tf.RaggedTensor [[b'a', b'v', b'c'],
 [b'v', b'v', b'c']]>

In [93]:
all_ids = ids_from_chars(tf.strings.unicode_split(all_text, "UTF-8"))
all_ids.shape

TensorShape([1115394])

In [94]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Prepare the dataset

In [95]:
ids_dataset = Dataset.from_tensor_slices(all_ids)

In [96]:
arr = next(iter(ids_dataset.batch(20).take(1)))
text_from_ids(arr)

<tf.Tensor: shape=(), dtype=string, numpy=b'First Citizen:\nBefor'>

In [103]:
arr = next(iter(ids_dataset.take(1)))
arr

<tf.Tensor: shape=(), dtype=int64, numpy=50>

In [106]:
def one_hot_encoding(Id):
    return tf.one_hot(Id, vocab_length)

In [107]:
ids_ohe_dataset = ids_dataset.map(one_hot_encoding)

In [108]:
seq_length = 100 + 1
ids_sequences = ids_ohe_dataset.batch(seq_length)

In [109]:
def split_input_target(sequence):
    input_seq = sequence[:-1]
    output_seq = sequence[1:]
    return input_seq, output_seq

In [113]:
dataset = ids_sequences.map(split_input_target)

Create batches for dataset

In [114]:
batch_size = 64
buffer_size = 10000
batched_dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

Prepare the model

In [184]:
class ZaraModelFirstLstmTry(tf.keras.Model):
    def __init__(self, hidden_size, vocab_size):
        super().__init__()
        self.lstm = layers.LSTM(hidden_size, return_sequences=True)
        self.dense = layers.Dense(vocab_size)

    def call(self, X, training=False):
        out = self.lstm(X, training=training)
        y = self.dense(out, training=training)
        return y

Create the model

In [185]:
hidden_size=128
model = ZaraModelFirstLstmTry(hidden_size, vocab_length)

In [186]:
for X, Y in batched_dataset.take(1):
    print(model(X).shape)

(64, 100, 65)
